# CISC7401 Homework 3 
## Deep Learning for Image Recognition 
@Author: Zhe He

Overall task: Given a training set of data, you should train a “deep model” (e.g., multi-layer
perceptron, convolutional neural network, and transformer), evaluate your model on the
testing set and some open-set data w.r.t some metrics, and write up a report.

Train different deep neural networks (including MLP, ResNet, ViT transformer) on the training
set. [This website and links therein](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) might be important to you.
You can refer to PyTorch for [building your network](https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html), or using other established networks such as
ResNet and Transformer. When training the deep neural networks for classification, you can use
cross-entropy loss.

See homework requirement for more details.

## Prepare phase
### First, before writing training code, let's define some helper function for training.

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import cm

from torch.utils.data import Dataset,ConcatDataset,DataLoader
from torch import nn, optim
import torch 
import copy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torchvision import transforms
import timm 
import timm.optim
from timm.data.transforms_factory import create_transform
from torch.optim import lr_scheduler
import os

sns.set_style("white")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
class MNISTdataset(Dataset):
    def __init__(self, data, transform = None):
        self.X = (data.drop('label', axis =1).values/255).reshape((-1, 28, 28,1))
        self.y = data['label'].values.reshape(-1,1)
        self.n_samples = data.shape[0]
        self.transform = transform 
    
    def __getitem__(self, index):
        X,y = self.X[index], self.y[index]
        if self.transform:
            X  = self.transform(X)
        return (X,y)
        
    def __len__(self):
        return self.n_samples

class MNISTdataset_inference(Dataset):
    """Inference"""
    def __init__(self, data, transform = None):
        self.X = (data.values/255).reshape((-1, 28, 28,1))
        self.n_samples = data.shape[0]
        self.transform = transform 
    
    def __getitem__(self, index):
        X = self.X[index]
        if self.transform:
            X  = self.transform(X)
        return X
        
    def __len__(self):
        return self.n_samples    


In [ ]:
#Train and Validation sets    
train_df = pd.read_csv('dataset/train.csv',dtype=np.float32)

## ResNet
We use a ResNet50 as our transfer learning model

In [ ]:
class confiFileResNet():
    """Configuration class for easy parametrization"""
    
    #Pretrained model with timm
    model = 'resnet50'
    epochs = 30
    
    in_chans = 1
    num_classes = 10
    learning_rate = 1e-3
    
    val_size = 0.3
    batch_size = 128
    
CFG_ResNet = confiFileResNet()

In [ ]:
resnet_transforms = {
    'original': transforms.Compose([transforms.ToTensor()]),
    'aug1': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=(300, 350), fill=0),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(28, scale=(0.8, 1.0))
    ]),
    'aug2': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=(0, 45), fill=0),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(28, scale=(0.8, 1.0))
    ]),
    'aug3': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=(-15, 15)),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(28, scale=(0.7, 1.3))
    ])
}

In [ ]:
# Helper Function
def train_(model, optmizer, loss_func, train_loader, device):
    """Function to train the model"""
    train_loss = 0.0
    train_correct = 0
    size_sampler = len(train_loader.sampler)
    
    for i, (images, labels) in enumerate(train_loader):
        
        # Pushing to device (cuda or CPU)
        images, labels = images.to(device), labels.to(device)
        
        #zeroing gradiants
        optmizer.zero_grad()
        
        #feedfoard
        y_hat = model(images)
        
        #Compute loss 
        loss = loss_func(y_hat, labels.long().squeeze())
        
        #Compute backpropagation
        loss.backward()
        
        #updating weights
        optmizer.step()
        
        # loss and correct values compute
        train_loss +=loss.item() * images.size(0)
        _ , pred = torch.max(y_hat.data, 1)
        train_correct +=sum(pred == labels.long().squeeze()).sum().item()
        
    return np.round(train_loss/size_sampler,4), np.round(train_correct*100./size_sampler,3)


def validation_(model, loss_func, val_loader, device):
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            y_hat = model(images)
            loss = loss_func(y_hat, labels.long().squeeze())
            val_loss += loss.item() * images.size(0)
            
            _, preds = torch.max(y_hat, dim=1)
            all_preds.append(preds)
            all_labels.append(labels.squeeze())
    
    all_preds = torch.cat(all_preds).cpu().numpy()
    all_labels = torch.cat(all_labels).cpu().numpy()
    
    val_loss = val_loss / len(val_loader.sampler)
    

    accuracy = accuracy_score(all_labels, all_preds) * 100  # 百分比形式，与原代码保持一致
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    
    return {
        'val_loss': np.round(val_loss, 4),
        'accuracy': np.round(accuracy, 3),
        'precision': np.round(precision, 4),
        'recall': np.round(recall, 4),
        'f1': np.round(f1, 4),
        'conf_matrix': conf_matrix
    }



def train_model(model, optmizer, loss_func, scheduler, train_loader, val_loader, epochs, device, log=True):
    best_acc = 0
    best_model = None
    print('Initializing Training...')

    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'val_precision': [],
        'val_recall': [],
        'val_f1': []
    }
    
    for i in range(epochs):
        train_loss, train_acc = train_(model, optmizer, loss_func, train_loader, device)
        val_metrics = validation_(model, loss_func, val_loader, device)
        scheduler.step()
        
        val_acc = val_metrics['accuracy']
        overfitting = train_acc - val_acc
        
        if val_acc > best_acc:
            print(f'>> Saving Best Model with Val Acc: Old: {best_acc} -> New: {val_acc}')
            best_model = copy.deepcopy(model)
            best_acc = val_acc
            best_metrics = val_metrics
            best_metrics['overfitting'] = overfitting
        
        if log and ((i + 1) % 2 == 0):
            print(f'> Epochs: {i+1}/{epochs} - Train Loss: {train_loss} - Train Acc: {train_acc} - '
                  f'Val Loss: {val_metrics["val_loss"]} - Val Acc: {val_acc} - '
                  f'Precision: {val_metrics["precision"]} - Recall: {val_metrics["recall"]} - '
                  f'F1: {val_metrics["f1"]}')

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_metrics['val_loss'])
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['val_precision'].append(val_metrics['precision'])
        history['val_recall'].append(val_metrics['recall'])
        history['val_f1'].append(val_metrics['f1'])
    
    print('Training Finished:)')
    return history, best_model, best_metrics

def get_model(model_type):
    if model_type == 'MLP':
        return timm.create_model('mixer_b16_224', pretrained=True, in_chans=3, num_classes=10).to(device)
    elif model_type == 'ResNet':
        return timm.create_model('resnet50', pretrained=True, in_chans=1, num_classes=10).to(device)
    elif model_type == 'ViT':
        return timm.create_model('vit_base_patch16_224', pretrained=True, in_chans=3, num_classes=10).to(device)
    else:
        raise ValueError("Unknown model type")

def plot_history(history):
    fig1, ax1 = plt.subplots(figsize=(8,6))
    sns.lineplot(data=history['train_loss'], label='Training Loss', ax=ax1)
    sns.lineplot(data=history['val_loss'], label='Validation Loss', ax=ax1)
    ax1.tick_params(axis='both', labelsize=32)
    ax1.set_xlabel('Epoch', fontsize=32)
    ax1.set_ylabel('Loss', fontsize=32)
    ax1.legend(loc='upper right', fontsize=32)
    fig1.savefig('loss_curve.png')
    plt.close(fig1)
    
    fig2, ax2 = plt.subplots(figsize=(8,6))
    sns.lineplot(data=history['train_acc'], label='Training Accuracy', ax=ax2)
    sns.lineplot(data=history['val_acc'], label='Validation Accuracy', ax=ax2)
    ax2.tick_params(axis='both', labelsize=32)
    ax2.set_xlabel('Epoch', fontsize=32)
    ax2.set_ylabel('Accuracy', fontsize=32)
    ax2.legend(loc='lower right', fontsize=32)
    fig2.savefig('accuracy_curve.png')
    plt.close(fig2)


## Vision Transformer
We use a pre trained ViT to transfer learning in MNIST

In [ ]:
class confiFileViT():
    model = 'vit_base_patch16_224'
    epochs = 30
    in_chans = 3
    num_classes = 10
    learning_rate = 3e-5
    val_size = 0.3
    batch_size = 64
CFG_ViT = confiFileViT()

In [ ]:
vit_transform_base = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

vit_transforms = {
    'original': vit_transform_base,
    'aug1': transforms.Compose([
        vit_transform_base,
        transforms.RandomRotation(degrees=(300, 350), fill=0),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0))
    ]),
    'aug2': transforms.Compose([
        vit_transform_base,
        transforms.RandomRotation(degrees=(0, 45), fill=0),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0))
    ]),
    'aug3': transforms.Compose([
        vit_transform_base,
        transforms.RandomRotation(degrees=(-15, 15)),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(224, scale=(0.7, 1.3))
    ])
}

## Multi Layer Perceptron
We use a pre trained MLP to do transfer learning 

In [ ]:
class confiFileMLP():
    model = 'mixer_b16_224'  # Pretrained MLP-Mixer model from timm
    pretrained_cfg = 'miil_in21k_ft_in1k'  # Pretrained configuration
    epochs = 30
    in_chans = 3            # MLP-Mixer requires 3-channel input
    num_classes = 10        # MNIST has 10 classes
    learning_rate = 5e-5    # Smaller learning rate suitable for MLP-Mixer
    img_size = 224          # MLP-Mixer requires 224x224 input
    batch_size = 256        # Batch size (adjust based on GPU memory)
    val_size = 0.3          # Validation set size

CFG_MLP = confiFileMLP()

mlp_transform_base = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

mlp_transforms = {
    'original': mlp_transform_base,
    'aug1': transforms.Compose([
        mlp_transform_base,
        transforms.RandomRotation(degrees=(300, 350), fill=0),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0))
    ]),
    'aug2': transforms.Compose([
        mlp_transform_base,
        transforms.RandomRotation(degrees=(0, 45), fill=0),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0))
    ]),
    'aug3': transforms.Compose([
        mlp_transform_base,
        transforms.RandomRotation(degrees=(-15, 15)),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomResizedCrop(224, scale=(0.7, 1.3))
    ])
}

# mixer_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Convert 1-channel to 3-channel
#     transforms.Resize((CFG_MLP.img_size, CFG_MLP.img_size)),  # Resize to 224x224
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet normalization
#                          std=[0.229, 0.224, 0.225]),
# ])

# compose_aug_mlp = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
#     transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
#     transforms.RandomResizedCrop(CFG_MLP.img_size, scale=(0.85, 1.0)),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225]),
# ])

# compose_original = mixer_transform

# dataset_original = MNISTdataset(train_df, transform=compose_original)
# dataset_aug = MNISTdataset(train_df, transform=compose_aug_mlp)

# increased_dataset = ConcatDataset([dataset_original, dataset_aug])

# train_df = pd.read_csv('dataset/train.csv', dtype=np.float32)

# X_train, X_val = train_test_split(increased_dataset, test_size=CFG_MLP.val_size, random_state=666)

# # Create data loaders
# train_loader = DataLoader(X_train, batch_size=CFG_MLP.batch_size, shuffle=True)
# val_loader = DataLoader(X_val, batch_size=CFG_MLP.batch_size, shuffle=True)

## Experiment

In [ ]:
train_df = pd.read_csv('dataset/train.csv', dtype=np.float32)
model_type = 'MLP'  # Change to 'MLP' or 'ViT' as needed
aug_strategy = 'original'  # Change to 'original', 'aug2', or 'aug3' as needed

# Select transforms based on model
if model_type == 'ResNet':
    transform = resnet_transforms[aug_strategy]
    val_transform = resnet_transforms['original']  # Validation set typically unaugmented
    batch_size = 128
    epochs = 30
    lr = 1e-3
elif model_type == 'ViT':
    transform = vit_transforms[aug_strategy]
    val_transform = vit_transforms['original']
    batch_size = 64
    epochs = 30
    lr = 3e-5
elif model_type == 'MLP':
    transform = mlp_transforms[aug_strategy]
    val_transform = mlp_transforms['original']
    batch_size = 256
    epochs = 30
    lr = 5e-5

In [ ]:
# Prepare datasets
train_dataset = MNISTdataset(train_df, transform=transform)
val_dataset = MNISTdataset(train_df, transform=val_transform)
train_set, val_set = train_test_split(train_dataset, test_size=0.3, random_state=666)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [ ]:
# Initialize model and training components
model = get_model(model_type)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
# Train and evaluate
history, best_model, best_metrics = train_model(
    model, optimizer, loss_func, scheduler, train_loader, val_loader, epochs, device
)

results = {
    'model': model_type,
    'aug_strategy': aug_strategy,
    'accuracy': best_metrics['accuracy'],
    'overfitting': best_metrics['overfitting'],
    'f1': best_metrics['f1'],
    'precision': best_metrics['precision'],
    'recall': best_metrics['recall']
}
results_df = pd.DataFrame([results])
results_df.to_csv(f'results_{model_type}_{aug_strategy}.csv', index=False)


test_transforms = {
    'ResNet': transforms.Compose([transforms.ToTensor()]),
    'ViT': transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'MLP': transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

#Test Sets
test_df = pd.read_csv('dataset/test.csv',dtype=np.float32)
test_dataset = MNISTdataset_inference(test_df, transform = test_transforms[model_type])
test_loader = DataLoader(test_dataset, batch_size=1024)

y_pred_list = []
with torch.no_grad():
    for X_test_ld in (test_loader):
        y_pred = best_model(X_test_ld.to(device))
        _, y_pred_tags = torch.max(y_pred.data, dim = 1)
#         y_pred_list.append((y_pred_tags.to('cpu')))

result = [int(i) for i in y_pred_list]
df = pd.DataFrame({'ImageId': range(1, len(result) + 1), 'Label': result})
output_filename = f'dataset/output_{model_type}_{aug_strategy}.csv'
df.to_csv(output_filename, index=False)
sub_status = os.path.exists(output_filename)

print(f"\Check for {model_type} with {aug_strategy}: {sub_status}")



# Save model and clear memory
torch.save(best_model.state_dict(), f'model_{model_type}_{aug_strategy}.pth')
torch.cuda.empty_cache()

print(f"Finished training {model_type} with {aug_strategy}")

In [ ]:
# Aggregate results
result_files = [f for f in os.listdir() if f.startswith('results_')]
all_results = pd.concat([pd.read_csv(f) for f in result_files], ignore_index=True)

# Save combined results
all_results.to_csv('ablation_study_results.csv', index=False)

# Plot augmentation strategy vs. performance
plt.figure(figsize=(12, 6))
for model in ['MLP', 'ResNet', 'ViT']:
    subset = all_results[all_results['model'] == model]
    plt.plot(subset['aug_strategy'], subset['accuracy'], marker='o', label=model)
plt.xlabel('Augmentation Strategy')
plt.ylabel('Validation Accuracy (%)')
plt.title('Accuracy Across Augmentation Strategies')
plt.legend()
plt.grid(True)
plt.savefig('accuracy_vs_augmentation.png')
plt.show()

# Display results table
print(all_results[['model', 'aug_strategy', 'accuracy', 'overfitting', 'f1', 'precision', 'recall']])